# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
!pip install yelpapi

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/scyjt/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s=5.0)
yelp

### Define Search Terms and File Paths

In [5]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98122'
term = 'pizza'

In [6]:
## Specify folder for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER,exist_ok=True)

location.split(',')[0]

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [7]:
JSON_FILE

'Data/Seattle-pizza.json'

### Check if Json File exists and Create it if it doesn't

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists ==False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w')as f:
        json.dump([],f)
        
## If it exists, inform user
else:
        print(f"[i] {JSON_FILE} already exists.")

[i] Data/Seattle-pizza.json already exists.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term=term,location=location)

In [10]:
## How many results total?
type(results)

dict

In [11]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
results['businesses']

[{'id': 't-Z9bvmlgUyDtomGmttrUQ',
  'alias': 'guerilla-pizza-kitchen-seattle',
  'name': 'Guerilla Pizza Kitchen',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEjc9J_4Tb1PtAb71UA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/guerilla-pizza-kitchen-seattle?adjust_creative=2VVn9tAdE7T7A_bSyPHVAg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2VVn9tAdE7T7A_bSyPHVAg',
  'review_count': 1,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'popuprestaurants', 'title': 'Pop-Up Restaurants'}],
  'rating': 5.0,
  'coordinates': {'latitude': 47.608127, 'longitude': -122.302435},
  'transactions': [],
  'location': {'address1': '2300 E Cherry St',
   'address2': '',
   'address3': None,
   'city': 'Seattle',
   'zip_code': '98122',
   'country': 'US',
   'state': 'WA',
   'display_address': ['2300 E Cherry St', 'Seattle, WA 98122']},
  'phone': '',
  'display_phone': '',
  'distance': 749.1731129459927},
 {'id': 'Wi6LFkjIa

In [13]:
results['total']

855

In [14]:
pd.DataFrame(results['businesses'])

id                                          alias  \
0   t-Z9bvmlgUyDtomGmttrUQ                 guerilla-pizza-kitchen-seattle   
1   Wi6LFkjIausYj277ru6pqg                             humble-pie-seattle   
2   EwrAoy7XbX8nfemZdDvupA                               cornelly-seattle   
3   M9xzvwgK58T0w7wvXedvuQ                        hot-mamas-pizza-seattle   
4   et4eWK49x5od4lfqCukG6w                          central-pizza-seattle   
5   FVbwpNA1uZEGiM02N4XtUg                                 blotto-seattle   
6   fxyHWmfzcdjImgQ_UYHoTw                              bar-cotto-seattle   
7   IrohtoYjnAR_vc6w6CRCxA                                   moto-seattle   
8   ugTsEtjvwRhteac_6JcuPw                   italian-family-pizza-seattle   
9   kVqwgAjEPffEyMJSzpfM_A                     pagliacci-pizza-seattle-26   
10  -FOAQv22SXtSBs7nptI3UA                 serious-pie-downtown-seattle-2   
11  n6BLr6spgjVWqzUwmVx0bg                    johnny-mos-pizzeria-seattle   
12  cAFXng3Gr1cBCFkP1VIxng             the-independent-pizzeria-seattle-2   
13  Ss_66zV-zhK_57_6MgV93w                           aglio-e-olio-seattle   
14  GJdmwqqkC0nAQfX1qZM3ew  olympia-pizza-and-spaghetti-house-iii-seattle   
15  wi-mbKEz4Ha1YYaOB8zHnw                             kōbo-pizza-seattle   
16  k3riaoflafifFUSNKXbfTw             via-tribunali-capitol-hill-seattle   
17  1rkgyfwzXG7qSdVZdfRt7g                                 roccos-seattle   
18  PpAiLhErwd08QhPb06_-hA                             mioposto-seattle-7   
19  HeJP0ZS0tCCn-X9ezYSz3g                ian-s-pizza-on-the-hill-seattle   

                                     name  \
0                  Guerilla Pizza Kitchen   
1                              Humble Pie   
2                                Cornelly   
3                        Hot Mama's Pizza   
4                           Central Pizza   
5                                  Blotto   
6                               Bar Cotto   
7                                    Moto   
8                    Italian Family Pizza   
9                         Pagliacci Pizza   
10                   Serious Pie Downtown   
11                   Johnny Mo's Pizzeria   
12               The Independent Pizzeria   
13                           Aglio e Olio   
14  Olympia Pizza and Spaghetti House III   
15                             Kōbo Pizza   
16           Via Tribunali - Capitol Hill   
17                                Rocco's   
18                               Mioposto   
19                Ian’s Pizza on the Hill   

                                            image_url  is_closed  \
0   https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...      False   
1   https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...      False   
2   https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...      False   
3   https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...      False   
4   https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...      False   
5   https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...      False   
6   https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...      False   
7   https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...      False   
8   https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...      False   
9   https://s3-media3.fl.yelpcdn.com/bphoto/7thHN-...      False   
10  https://s3-media2.fl.yelpcdn.com/bphoto/dy0pJ5...      False   
11  https://s3-media4.fl.yelpcdn.com/bphoto/8IAnDP...      False   
12  https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...      False   
13  https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...      False   
14  https://s3-media1.fl.yelpcdn.com/bphoto/KvTgMA...      False   
15  https://s3-media1.fl.yelpcdn.com/bphoto/6et4T9...      False   
16  https://s3-media4.fl.yelpcdn.com/bphoto/8nqcG7...      False   
17  https://s3-media3.fl.yelpcdn.com/bphoto/AWBXTE...      False   
18  https://s3-media1.fl.yelpcdn.com/bphoto/gWBw_l...      False   
19  https://s3-media1.fl.yelpcdn.com/bphoto/gD-SKu...      False   

                               

- Where is the actual data we want to save?

In [15]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [16]:
# Use math.ceil to round up for the total number of pages of results.
import time, math
n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

43

In [30]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:     
        time.sleep(.2)
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location-location,
                                   term=term,
                                   offset=n_pages+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)

    ## What to do if we get an error/exception.
    except Exception as e:
        print('[!] ERROR: ',e)


  0%|          | 0/43 [00:00<?, ?it/s]

[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' and 'str'
[!] ERROR:  unsupported operand type(s) for -: 'str' an

## Open the Final JSON File with Pandas

In [ ]:
df = None

In [ ]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

In [ ]:
## Save it as a compressed csv (to save space)


## Bonus: compare filesize with os module's `os.path.getsize`

In [ ]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

## Next Class: Processing the Results and Mapping 